# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

## LLM Provider Configuration

You can switch between **Ollama** (local, free) and **OpenAI** (cloud API).

**To change:**
1. Edit the next cell
2. Change `USE_PROVIDER = 'ollama'` to `USE_PROVIDER = 'openai'`
3. Execute cells from here downward

### Cost Comparison:

| Provider | Model | Input Cost | Output Cost | Speed |
|----------|-------|------------|-------------|-------|
| **Ollama** | deepseek-v3.1:671b-cloud | FREE | FREE | Fast (local) |
| **OpenAI** | gpt-4o-mini | $0.15 / 1M tokens | $0.60 / 1M tokens | Very fast (cloud) |

**Estimate for this notebook:**
- With OpenAI: ~$0.01 - $0.05 per complete execution
- With Ollama: $0.00 (100% free)

In [2]:
# CONFIGURATION: Change here to test different providers
# Options: 'ollama' or 'openai'
USE_PROVIDER = 'ollama'  # Change to 'openai' to use OpenAI

print(f"Selected provider: {USE_PROVIDER.upper()}")

Selected provider: OLLAMA


In [3]:
# Initialize and constants

# Load configuration from global .env
load_dotenv(dotenv_path='/workspace/.env', override=True)

# Use provider selected in previous cell (or from .env as fallback)
llm_provider = USE_PROVIDER if 'USE_PROVIDER' in globals() else os.getenv('LLM_PROVIDER', 'ollama')

if llm_provider == 'ollama':
    # OLLAMA CONFIGURATION (Local)
    ollama_base_url = os.getenv('OLLAMA_BASE_URL')
    ollama_api_key = os.getenv('OLLAMA_API_KEY')
    ollama_model = os.getenv('OLLAMA_MODEL')
    
    print("=" * 50)
    print("Using OLLAMA (Local)")
    print("=" * 50)
    print(f"   Base URL: {ollama_base_url}")
    print(f"   Model: {ollama_model}")
    print(f"   Cost: FREE")
    print("=" * 50)
    
    # Create OpenAI client pointing to Ollama
    openai = OpenAI(
        base_url=f"{ollama_base_url}/v1",
        api_key=ollama_api_key
    )
    MODEL = ollama_model
    
else:
    # OPENAI CONFIGURATION (Cloud)
    api_key = os.getenv('OPENAI_API_KEY')
    
    print("=" * 50)
    print("Using OPENAI (Cloud API)")
    print("=" * 50)
    
    if api_key and api_key.startswith('sk-proj-') and len(api_key) > 50:
        print(f"   API Key: sk-proj-...{api_key[-8:]}")
        print(f"   Model: gpt-4o-mini")
        print(f"   Cost: ~$0.15 / 1M tokens input")
        print("   Status: Configured correctly")
    else:
        print("   Status: Invalid or missing API Key")
        print("   Check the .env file")
    
    print("=" * 50)
    
    openai = OpenAI(api_key=api_key)
    MODEL = 'gpt-4o-mini'

Using OLLAMA (Local)
   Base URL: http://192.168.80.200:11434
   Model: deepseek-v3.1:671b-cloud
   Cost: FREE


In [4]:
# Clase para representar una pagina web con BeautifulSoup

import requests
from bs4 import BeautifulSoup

class WebPage:
    """Class to represent a scraped webpage, with links and content."""
    
    def __init__(self, url):
        """
        Constructor: Downloads and parses a webpage.
        
        Args:
            url (str): URL of the webpage to scrape
        """
        self.url = url
        
        # 1. Make HTTP request and get content
        response = requests.get(url)
        self.body = response.content  # Fixed: was 'boby'
        
        # 2. Parse HTML with BeautifulSoup
        self.soup = BeautifulSoup(self.body, 'html.parser')  # Fixed: missing 'self.'
        
        # 3. Extract page title
        self.title = self.soup.title.string if self.soup.title else 'No Title'
        
        # 4. Clean and extract body text
        if self.soup.body:
            # Remove irrelevant elements (scripts, styles, navigation, etc.)
            for irrelevant in self.soup.body(['script', 'style', 'header', 'footer', 'nav', 'aside']):
                irrelevant.decompose()
            # Get all cleaned text
            self.text = self.soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = 'No Body Content'
        
        # 5. Extract all links (href) from page
        links = [link.get('href') for link in self.soup.find_all('a', href=True)]
        # Filter out empty or None links
        self.links = [link for link in links if link]
        
    def get_contents(self):
        """
        Returns page content in readable format.
        
        Returns:
            str: Title and text content of the page
        """
        return f"Title: {self.title}\n\nContent:\n{self.text}"

## Connection Test

Run this cell to verify that the selected provider works correctly.

In [5]:
# Quick connection test
print(f"Testing connection with {llm_provider.upper()}...\n")

try:
    test_response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": "Respond only with: 'Connection successful!'"}
        ],
        max_tokens=50
    )
    result = test_response.choices[0].message.content
    print(f"SUCCESS!")
    print(f"Model response: {result}")
    print(f"Model used: {MODEL}")
except Exception as e:
    print(f"Connection ERROR:")
    print(f"   {str(e)}")

Testing connection with OLLAMA...

SUCCESS!
Model response: Connection successful!
Model used: deepseek-v3.1:671b-cloud


---

## First step: Have gpt-4o-mini or deepseek-v3.1:671b-cloud figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.

IMPORTANT: You MUST respond with ONLY valid JSON, no other text before or after. Do not include markdown code blocks.

Respond in this exact JSON format:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)


You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.

IMPORTANT: You MUST respond with ONLY valid JSON, no other text before or after. Do not include markdown code blocks.

Respond in this exact JSON format:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(webpage):
    """
    Generates user prompt with the list of links from a webpage.
    
    Args:
        webpage (WebPage): WebPage object with extracted links
    
    Returns:
        str: Formatted prompt with link list
    """
    user_prompt = f"""
Here is the list of links on the website {webpage.url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    # Use links directly from WebPage object (already extracted)
    user_prompt += "\n".join(webpage.links)
    return user_prompt

In [9]:
def select_relevant_links(webpage):
    """
    Selects relevant links from a webpage using GPT/Ollama.
    
    Args:
        webpage (WebPage): WebPage object with extracted links
    
    Returns:
        dict: JSON with relevant links in format {"links": [...]}
    """
    # NOTE: Ollama does not support response_format, so we make it conditional
    if llm_provider == 'ollama':
        # For Ollama: without response_format, rely on prompt
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": link_system_prompt},
                {"role": "user", "content": get_links_user_prompt(webpage)}
            ],
            temperature=0  # More deterministic for consistent JSON
        )
    else:
        # For OpenAI: use response_format
        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": link_system_prompt},
                {"role": "user", "content": get_links_user_prompt(webpage)}
            ],
            response_format={"type": "json_object"}
        )
    
    result = response.choices[0].message.content
    
    # CLEANUP: Remove markdown code blocks if present
    # Some models wrap JSON in ```json ... ```
    if result.strip().startswith('```'):
        # Extract content between ``` markers
        lines = result.strip().split('\n')
        # Remove first line (```json or ```) and last line (```)
        result = '\n'.join(lines[1:-1])
    
    # Attempt to parse JSON with error handling
    try:
        links = json.loads(result)
        print(f"Found {len(links.get('links', []))} relevant links")
        return links
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON. Model response:")
        print(result)
        print(f"\nError: {e}")
        # Return empty structure on error
        return {"links": []}
    

In [10]:
antropic = WebPage("https://anthropic.com")
antropic.links

['#main',
 '#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/news/claude-sonnet-4-5',
 'https://www.anthropic.com/news/claude-haiku-4-5',
 'https://anthropic.com/news/context-management',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/news/core-views-on-ai-safety',
 'https://www.anthropic.com/rsp-updates',
 'https://www.anthropic.com/learn',
 'https://www.anthropic.com/news/claude-haiku-4-5',
 'https://www.anthropic.com/news/claude-sonnet-4-5',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/news/claude-opus-4-1',
 'https://www.anthropic.com/research/project-vend-1',
 'https://www.anthropic.com/research/agentic-misalignment',
 'https://www.anthropic.com/research/tracing-thoughts-language-model',
 'https://www.anthropic.com/jobs',
 'https://www.anthropic.com/jobs',
 'https://www.anthropic.com/contact-sales']

In [11]:
select_relevant_links(antropic)

Found 10 relevant links


{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/'},
  {'type': 'news page',
   'url': 'https://www.anthropic.com/news/claude-sonnet-4-5'},
  {'type': 'news page',
   'url': 'https://www.anthropic.com/news/claude-haiku-4-5'},
  {'type': 'news page',
   'url': 'https://anthropic.com/news/context-management'},
  {'type': 'product page', 'url': 'https://www.anthropic.com/claude/sonnet'},
  {'type': 'research page',
   'url': 'https://www.anthropic.com/research/project-vend-1'},
  {'type': 'research page',
   'url': 'https://www.anthropic.com/research/agentic-misalignment'},
  {'type': 'research page',
   'url': 'https://www.anthropic.com/research/tracing-thoughts-language-model'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/jobs'},
  {'type': 'contact page', 'url': 'https://www.anthropic.com/contact-sales'}]}

In [12]:
# Probar con HuggingFace
hf = WebPage("https://huggingface.co")
select_relevant_links(hf)

Found 7 relevant links


{'links': [{'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'join page', 'url': 'https://huggingface.co/join'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [ ]:
def fetch_page_and_all_relevant_links(url):
    """
    Fetches content from main page and all its relevant links.
    
    Args:
        url (str): Website URL
    
    Returns:
        str: Formatted content with main page and relevant links
    """
    # Create WebPage object (download and parse ONCE)
    webpage = WebPage(url)
    
    # Get main page content
    contents = webpage.get_contents()
    
    # Select relevant links using GPT
    relevant_links = select_relevant_links(webpage)
    
    # Build result
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

In [ ]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,  # FIXED: Use MODEL variable (gpt-4o-mini or ollama)
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,  # FIXED: Use MODEL variable (gpt-4o-mini or ollama)
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>